In [ ]:
# EN:
# If you run this in a fresh Colab, uncomment the line below:
# RU:
# Если запускаешь в чистом Colab, раскомментируй строку ниже:

# !pip install -q gra-core

import numpy as np
import matplotlib.pyplot as plt

from gra_core import GraNullifier
from gra_core.utils import homogeneous_projector, make_level_dict


In [ ]:
# EN:
# Toy multiverse setup:
# - 2 levels: l = 0 and l = 1
# - at each level: N random 4D states
# - same simple projector at both levels (onto [1, 1, 1, 1])
#
# RU:
# Игрушечный мультиверс:
# - 2 уровня: l = 0 и l = 1
# - на каждом уровне: N случайных 4-мерных состояний
# - один и тот же простой проектор для обоих уровней (на [1, 1, 1, 1])

dim = 4
n_per_level = 5
levels = [0, 1]

psi_init = make_level_dict(levels=levels, dim=dim, n_per_level=n_per_level, seed=42)

P = homogeneous_projector(dim=dim)
projectors = {0: P, 1: P}


In [ ]:
# EN:
# Helper to compute Phi^{(l)} for visualization:
#   Phi^{(l)} = sum_{i != j} | <psi_i | P | psi_j> |^2
#
# RU:
# Вспомогательная функция для вычисления Phi^{(l)}:
#   Phi^{(l)} = sum_{i != j} | <psi_i | P | psi_j> |^2

def phi_level(psi_level, projector):
    phi = 0.0
    L = len(psi_level)
    if L <= 1:
        return 0.0
    for i in range(L):
        v_i = projector(psi_level[i])
        for j in range(L):
            if i == j:
                continue
            v_j = projector(psi_level[j])
            amp = np.vdot(v_i, v_j)
            phi += float(np.abs(amp) ** 2)
    return phi


In [ ]:
# EN:
# Run GraNullifier and track Phi^{(0)} and Phi^{(1)} over iterations
# by manually unrolling the inner loop (for clearer plots).
#
# RU:
# Запускаем GraNullifier и отслеживаем Phi^{(0)} и Phi^{(1)} по итерациям,
# вручную разворачивая внутренний цикл (для более наглядного графика).

nullifier = GraNullifier(
    levels=1,
    alpha=0.9,
    lambda0=1.0,
    lr=0.05,
    eps=1e-6,
    max_iter=200,
)

# EN: copy initial states to avoid in-place modification
# RU: копируем начальные состояния, чтобы не портить psi_init
psi = {l: [v.copy() for v in vs] for l, vs in psi_init.items()}

phi_history_level0 = []
phi_history_level1 = []

for it in range(nullifier.max_iter):
    phi0 = phi_level(psi[0], projectors[0])
    phi1 = phi_level(psi[1], projectors[1])
    phi_history_level0.append(phi0)
    phi_history_level1.append(phi1)

    if phi0 < nullifier.eps and phi1 < nullifier.eps:
        print(f"EN: Converged at iteration {it}")
        print(f"RU: Сходимость достигнута на итерации {it}")
        break

    # EN: one pseudo-gradient step per level
    # RU: один псевдоградиентный шаг на уровень
    for l in [0, 1]:
        P_l = projectors[l]
        updated = []
        for v in psi[l]:
            v_proj = P_l(v)
            # EN: move towards projector subspace
            # RU: двигаемся в сторону подпространства проектора
            v_new = v - nullifier.lr * (v - v_proj)
            v_new = v_new / (np.linalg.norm(v_new) + 1e-12)
            updated.append(v_new)
        psi[l] = updated


In [ ]:
# EN:
# Plot Phi^{(0)} and Phi^{(1)} vs iteration (log scale for clarity).
#
# RU:
# Рисуем Phi^{(0)} и Phi^{(1)} по итерациям (логарифмический масштаб).

plt.figure(figsize=(8, 4))
plt.plot(phi_history_level0, label="Phi^(0)")
plt.plot(phi_history_level1, label="Phi^(1)")
plt.yscale("log")
plt.xlabel("Iteration / Итерация")
plt.ylabel("Foam Phi^(l) (log) / Пена Phi^(l) (лог)")
plt.title("GRA Multiverse Nullification: Foam decay / Затухание пены на двух уровнях")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# EN:
# Compare initial vs final projections onto the homogeneous direction
# to show that states become more "aligned" after nullification.
#
# RU:
# Сравниваем начальные и финальные проекции на однородное направление,
# чтобы увидеть, что после обнуления состояния становятся более «согласованными».

u = np.ones(dim)
u = u / (np.linalg.norm(u) + 1e-12)

def projections_on_u(states):
    return [float(np.dot(v, u)) for v in states]

init_proj_l0 = projections_on_u(psi_init[0])
final_proj_l0 = projections_on_u(psi[0])

print("EN: Level 0 projections onto [1,1,1,1]/||.||")
print("RU: Проекции уровня 0 на [1,1,1,1]/||.||")
print("EN: Initial:", init_proj_l0)
print("RU: Начальные:", init_proj_l0)
print("EN: Final:  ", final_proj_l0)
print("RU: Финальные:", final_proj_l0)
